In [1]:
# Install Packages
# !pip install pymongo
# !pip freeze

In [2]:
# Import Libraries
import pymongo
import json
from bson.objectid import ObjectId
from datetime import datetime
import pandas as pd

In [3]:
#Establish the connection and make the database
client = pymongo.MongoClient("mongodb://localhost:27017/") 
db_list = client.list_database_names()
if "Tweets_DB" in db_list:
    print("Database already exists. Will be dropped...")
    client.drop_database("Tweets_DB")
db = client["Tweets_DB"]
col_list = db.list_collection_names()
if "Tweets_data" in col_list:
        print("Tweets Collection already exists. Will be dropped...")
        db.Tweets_data.drop()
tweets_collec = db["Tweets_data"]
print("Database connection established")

Database already exists. Will be dropped...
Database connection established


In [4]:
def insert_tweet(tweet_info):
    try:
        tweets_collec.insert_one(tweet_info)
    except Exception as e:
        print(str(e))

In [5]:
def find_tweet(tweet_id):
    tweet = tweets_collec.find_one({"Tweet_Id": tweet_id})
    if tweet:
        return True
    else:
        return False

In [6]:
def retweet_counter(tweet_id):
    tweets_collec.update({'Tweet_Id': tweet_id}, {'$inc': {'Retweet_Count': 1}})

In [7]:
with open('corona-out-2', "r") as read_file:
    for line in read_file:
        try:
            data = json.loads(line)
            
            # check if tweet already exists
            if find_tweet(data['id_str']):
                print("tweet alreay exists")
                continue
            
            # check for retweet
            if data['text'].startswith('RT'):
                if data['retweeted_status']!=None:
                    Retweet_Id = data['retweeted_status']['id_str']
                    if find_tweet(Retweet_Id):
                        retweet_counter(Retweet_Id)
                    else:
                        retweet_info = {
                            'created_at':datetime.strftime(datetime.strptime(data['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'),
                            'Tweet_Id':data['retweeted_status']['id_str'],
                            'Text':data['retweeted_status']['text'],
                            'Hashtag':list(map(lambda x: x["text"], data['retweeted_status']['entities']['hashtags'])),
                            'User_Id':data['retweeted_status']['user']['id_str'],
                            'Retweet_Count': 1,
                            'Retweet_Id': 0
                        }
                        insert_tweet(retweet_info)
            else:
                Retweet_Id = 0
                
            tweet_info = {
                'created_at':datetime.strftime(datetime.strptime(data['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'),
                'Tweet_Id':data['id_str'],
                'Text':data['text'],
                'Hashtag':list(map(lambda x: x["text"], data['entities']['hashtags'])),
                'User_Id':data['user']['id_str'],
                'Retweet_Count': 0,
                'Retweet_Id':Retweet_Id
            }

            insert_tweet(tweet_info)

        except:
            continue


tweet alreay exists
tweet alreay exists
tweet alreay exists
tweet alreay exists
tweet alreay exists
tweet alreay exists
tweet alreay exists


In [8]:
client.list_database_names()

['Tweets_DB', 'admin', 'config', 'local']

In [9]:
tweets_collec.count_documents({})

14625